In [333]:
#Import libraries we need
import pandas as pd
import numpy as np
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [334]:
#Loading the beer relative datasets, and concat those datasets 
beer = pd.read_csv("beer_profile_and_ratings.csv")
price = pd.read_csv("makeup_beer_price.csv",encoding= 'unicode_escape')
price = price['made up price']
beer = pd.concat([beer,price], axis = 1)

In [335]:
#Constraints: All number of reviews need to large than 25% of dataset’s.
num_reviews_25 = np.quantile(beer['number_of_reviews'], 0.25)
beer = beer[beer['number_of_reviews']>num_reviews_25]

In [336]:
#Cleaning up our dataset
beer = beer.dropna()
beer_data = beer.drop(['Name', 'Style', 'Brewery', 'Beer Name (Full)', 'Description', 'Min IBU', 'Max IBU', 'review_appearance', 'review_palate', 'made up price'], axis =1)
beer_data_contain = beer_data.drop(['review_aroma', 'review_taste', 'review_overall', 'number_of_reviews'], axis = 1)

In [337]:
#Transfer our data type from dataframe to list
beer_data_contain = beer_data_contain.values.tolist()
name = beer['Name'].tolist()
style = beer['Style'].tolist()
abv = beer['ABV'].tolist()
astringency = beer['Astringency'].tolist()
body = beer['Body'].tolist()
alcohol = beer['Alcohol'].tolist()
bitter = beer['Bitter'].tolist()
sweet = beer['Sweet'].tolist()
sour = beer['Sour'].tolist()
salty = beer['Salty'].tolist()
fruits = beer['Fruits'].tolist()
hoppy = beer['Hoppy'].tolist()
spices = beer['Spices'].tolist()
malty = beer['Malty'].tolist()
aroma = beer['review_aroma'].tolist()
taste = beer['review_taste'].tolist()
overall = beer['review_overall'].tolist()
price = price.tolist()

In [338]:
#Style constraint: each style (111 styles) of beer need at least one.
#As a result, we need to first find out the location of each group. 
temp_style = ' '
cutoff =[]
for s in range(len(style)):
    if style[s] != temp_style :
        cutoff.append(s)
    temp_style = style[s]
cutoff.append(len(style))

In [339]:
#Setting up the constraint relative variables and data 
num_style = len(beer.groupby('Style').count())
abv_25 = np.quantile(abv, 0.25)
abv_75 = np.quantile(abv, 0.75)
astringency_25 = np.quantile(astringency, 0.25)
astringency_75 = np.quantile(astringency, 0.75)
body_25 = np.quantile(body, 0.25)
body_75 = np.quantile(body, 0.75)
alcohol_25 = np.quantile(alcohol, 0.25)
alcohol_75 = np.quantile(alcohol, 0.75)
bitter_25 = np.quantile(bitter, 0.25)
bitter_75 = np.quantile(bitter, 0.75)
sweet_25 = np.quantile(sweet, 0.25)
sweet_75 = np.quantile(sweet, 0.75)
sour_25 = np.quantile(sour, 0.25)
sour_75 = np.quantile(sour, 0.75)
salty_25 = np.quantile(salty, 0.25)
salty_75 = np.quantile(salty, 0.75)
fruits_25 = np.quantile(fruits, 0.25)
fruits_75 = np.quantile(fruits, 0.75)
hoppy_25 = np.quantile(hoppy, 0.25)
hoppy_75 = np.quantile(hoppy, 0.75)
spices_25 = np.quantile(spices, 0.25)
spices_75 = np.quantile(spices, 0.75)
malty_25 = np.quantile(malty, 0.25)
malty_75 = np.quantile(malty, 0.75)
aroma_mean = np.mean(aroma)
taste_mean = np.mean(taste)
data_25 = [abv_25, astringency_25, body_25, alcohol_25, bitter_25, sweet_25, sour_25, salty_25, fruits_25, hoppy_25, spices_25, malty_25]
data_75 = [abv_75, astringency_75, body_75, alcohol_75, bitter_75, sweet_75, sour_75, salty_75, fruits_75, hoppy_75, spices_75, malty_75]

In [340]:
#Modeling
model = ConcreteModel()

#Input variables
num_beer = len(beer) #i 
num_contain = len(data_25) #j
bottle_upperlimit = 50
bottle_lowerlimit = 5
bottle_minimum = 3000
budget = 500000

#Generate our decision variables
model.x = Var(range(num_beer), domain = NonNegativeIntegers)
model.y = Var(range(num_beer), domain = Binary)


#Average AVB, Astringency, Body, Alcohol, Bitter, Sweet, Sour, Salty, Fruits, Hoppy, Spices, Malty need to be between 25% and 75% of the dataset’s. 
model.contain = ConstraintList()
for j in range(num_contain):
    model.contain.add(expr = sum(model.x[i] * beer_data_contain[i][j] for i in range(num_beer)) >= data_25[j]*sum(model.x[k] for k in range(num_beer)))
    model.contain.add(expr = sum(model.x[i] * beer_data_contain[i][j] for i in range(num_beer)) <= data_75[j]*sum(model.x[k] for k in range(num_beer)))

#Each kind of beer can at most import 50 bottles    
model.bottle_upperlimit = ConstraintList()
for i in range(num_beer):
    model.bottle_upperlimit.add(expr = model.x[i] - (bottle_upperlimit * model.y[i]) <= 0)
    
#If we want to purchase, must purchase more than 5 bottles
model.bottle_lowerlimit = ConstraintList()
for i in range(num_beer):
    model.bottle_lowerlimit.add(expr = model.x[i] - (bottle_lowerlimit * model.y[i]) >= 0)

#Each style (111 styles) of beer need at least one.
model.style = ConstraintList()
for z in range(len(cutoff)-1):
    model.style.add(expr = sum(model.x[i] for i in range(cutoff[z],cutoff[z+1]))>=1)

#Average review aroma and review taste rate need to be larger than 50% of dataset’s.    
model.Constraint_aroma = Constraint(expr = sum(model.x[i] * aroma[i] for i in range(num_beer)) >= aroma_mean*sum(model.x[k] for k in range(num_beer)))
model.Constraint_taste = Constraint(expr = sum(model.x[i] * taste[i] for i in range(num_beer)) >= taste_mean*sum(model.x[k] for k in range(num_beer)))

#Budget constraint
model.Constraint_price = Constraint(expr = sum(model.x[i] * price[i] for i in range(num_beer)) <= budget)

#Average overall rate >= 4
total_overall = sum(model.x[i] * overall[i] for i in range(num_beer))
total_bottle = sum(model.x[k] for k in range(num_beer))
model.Constraint_average_overall = Constraint(expr = total_overall >= 4*total_bottle)

#Bottle Minimum constraint
model.constraint_bottle_minimum = Constraint(expr = sum(model.x[i] for i in range(num_beer)) >= bottle_minimum  )

#Objective : Minimize Cost
total_cost = sum(model.x[i] * price[i] for i in range(num_beer))
model.Objective = Objective(expr = total_cost, sense = minimize)


In [341]:
#Optimize and print the total cost and average overall rate
opt = SolverFactory('glpk')
opt.options['mipgap'] = 0.02
results = opt.solve(model, tee = True)
print("Obj: Total Cost:", model.Objective())
print("Average Overall Rate", total_overall()/total_bottle())

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --mipgap 0.02 --write /var/folders/n9/t_j1751j2636q1vf1wsyppsw0000gn/T/tmp628xec54.glpk.raw
 --wglp /var/folders/n9/t_j1751j2636q1vf1wsyppsw0000gn/T/tmpswfb7bf5.glpk.glp
 --cpxlp /var/folders/n9/t_j1751j2636q1vf1wsyppsw0000gn/T/tmpl350xq0p.pyomo.lp
Reading problem data from '/var/folders/n9/t_j1751j2636q1vf1wsyppsw0000gn/T/tmpl350xq0p.pyomo.lp'...
/var/folders/n9/t_j1751j2636q1vf1wsyppsw0000gn/T/tmpl350xq0p.pyomo.lp:102181: warning: lower bound of variable 'x2385' redefined
/var/folders/n9/t_j1751j2636q1vf1wsyppsw0000gn/T/tmpl350xq0p.pyomo.lp:102181: warning: upper bound of variable 'x2385' redefined
4909 rows, 4769 columns, 77908 non-zeros
4768 integer variables, 2384 of which are binary
104565 lines were read
Writing problem data to '/var/folders/n9/t_j1751j2636q1vf1wsyppsw0000gn/T/tmpswfb7bf5.glpk.glp'...
97267 lines were written
GLPK Integer Optimizer 5.0
4909 rows, 4769 columns, 77908 non-zeros
4768 intege

In [346]:
#Print out the optimial import strategy
result_name = []
result_style = []
result_amount = []
result_unit_price = []
result_total_price= []
result_overall = []
for i in range(num_beer):
    if model.x[i]() > 0:
        result_name.append(name[i])
        result_style.append(style[i])
        result_amount.append(model.x[i]())
        result_unit_price.append(price[i])
        result_total_price.append(price[i]*model.x[i]())
        result_overall.append(overall[i])
        print('Name:', name[i],'Style: ', style[i],'Purchase', model.x[i](), 'bottles')

Name: Double Bag Style:  Altbier Purchase 5.0 bottles
Name: Gratitude Style:  Barleywine - American Purchase 5.0 bottles
Name: Bourbon Barrel-Aged Barley Wine Style:  Barleywine - English Purchase 5.0 bottles
Name: Bachelor ESB Style:  Bitter - English Extra Special / Strong Bitter (ESB) Purchase 50.0 bottles
Name: The Naughty Nurse Style:  Bitter - English Purchase 5.0 bottles
Name: Malheur Bière Brut (Brut Reserve) Style:  Bière de Champagne / Bière Brut Purchase 5.0 bottles
Name: Citra Blonde Summer Brew Style:  Blonde Ale - American Purchase 50.0 bottles
Name: De Koninck Blond Style:  Blonde Ale - Belgian Purchase 5.0 bottles
Name: Spaten Optimator Style:  Bock - Doppelbock Purchase 50.0 bottles
Name: Ramstein Winter Wheat Eisbock Style:  Bock - Eisbock Purchase 5.0 bottles
Name: Narragansett Bock Style:  Bock - Maibock Purchase 5.0 bottles
Name: St. Nikolaus Bock Bier Style:  Bock - Traditional Purchase 50.0 bottles
Name: Moonglow Weizenbock Style:  Bock - Weizenbock Purchase 50.0

In [348]:
#Store the result as a CSV file (Purchase_Beer_Strategy.csv)
d = {'Name':result_name,'Style':result_style, 'Unit Price':result_unit_price, 'Amount':result_amount, 'Total Price': result_total_price, 'Overall Rate': result_overall}
beer_import_data = pd.DataFrame(data= d)
beer_import_data.to_csv('Purchase_Beer_Strategy.csv', index=False)